# Quasi static fracture simulations

## Basic usage

The `matscipy.fracture_mechanics.crack` module provides classes and functions for setting up and analysing atomistic simulations of fracture.

In [45]:
import numpy as np

import ase.units as units
from ase.build import bulk
from ase.constraints import ExpCellFilter
from ase.optimize.precon import PreconLBFGS

from matscipy.fracture_mechanics.clusters import diamond, set_groups
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber, Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody

We first setup a manybody calculator using the "inadvertendly modified Stillinger Weber" (IMSW) parameters, in which the strength of the 3-body potential was (initially accidentally) doubled. This is useful for us here as it gives a brittle fracture response.

In [46]:
calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si)a)

### Bulk and Elastic properties

We can use this calculator to find the equilibrium lattice constant for bulk silicon by performing a variable cell relaxation using [ExpCellFilter](https://wiki.fysik.dtu.dk/ase/ase/constraints.html#the-expcellfilter-class) filter and [PreconLBFGS](https://wiki.fysik.dtu.dk/ase/ase/optimize.html#preconditioned-optimizers) optimizer from ASE. Since the cell is very small the preconditioning does not speed up the calculation; however, it is still useful as this is one of the most robust optimizers and it converges quickly. It is not necessary to use a cubic cell, we do it simply to make the calcualation of the cubic lattice constant `alat` straight-forward.

In [47]:
el = 'Si' # chemical element
si = bulk(el, cubic=True)
si.calc = calc
ecf = ExpCellFilter(si)
opt = PreconLBFGS(ecf)
opt.run(fmax=1e-6, smax=1e-6)

PreconLBFGS:   0  15:31:08      -34.692786       0.0000       0.0003


/Users/u1470235/gits/matscipy/venv/lib/python3.10/site-packages/ase/optimize/precon/lbfgs.py:133: UserWarning: The system is likely too small to benefit from the standard preconditioner, hence it is disabled. To re-enable preconditioning, call `PreconLBFGS` by explicitly providing the kwarg `precon`
  warnings.warn('The system is likely too small to benefit from '


PreconLBFGS:   1  15:31:08      -34.692800       0.0000       0.0000


True

In [48]:
a0 = si.cell[0, 0]

Next we compute the elastic constants using `matscipy.elasticity.fit_elastic_constants()` assuming cubic symmetry to reduce the number of calculations required.

In [49]:
from matscipy.elasticity import fit_elastic_constants

si = bulk(el, a=a0)
si.calc = calc
C, C_err = fit_elastic_constants(si, symmetry='cubic')

Fitting C_11
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-4.18433656e+00, -2.05309496e+00,  1.32979590e-05,  1.97563406e+00,
        3.87437477e+00]) GPa
Cij (gradient) / GPa    :     201.4615167357166
Error in Cij / GPa      :     2.647086489317546
Correlation coefficient :     0.9997411342093212
Setting C11 (1) to 1.257424 +/- 0.016522


Fitting C_21
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-1.13682864e+00, -5.40632635e-01,  1.32979590e-05,  4.89079644e-01,
        9.30322868e-01]) GPa
Cij (gradient) / GPa    :     51.640152889484874
Error in Cij / GPa      :     1.7644310200757045
Correlation coefficient :     0.9982534274183545
Setting C21 (7) to 0.322312 +/- 0.011013


Fitting C_31
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-1.13682864e+00, -5.40632635e-01,  1.32979590e-05,  4.89079644e-01,
        9.30322868e-01]) GPa
Cij (gradient) / GPa    :     51.64015288948469
Error in Cij / GPa      :     1.764431020075586
Corr

In [41]:
C_11 = C[0, 0] / units.GPa
C_12 = C[0, 1] / units.GPa
C_44 = C[3, 3] / units.GPa

We extract the cubic elastic constants $C_{11}$, $C_{12}$ and $C_{44}$ and convert to units of GPa:

In [43]:
(C_11, C_12, C_44)

(201.4615167357166, 51.64015288948479, 118.17710649364383)

The remaining ingredient is the surface energy, which we can also compute on-the-fly

In [52]:
# UPDATE to import from here once `adapticecont` branch is merged.
# from matscipy.fracture_mechanics.clusters import find_surface_energy

def find_surface_energy(symbol,calc,a0,surface,size=(8,1,1),vacuum=10,fmax=0.0001,unit='0.1J/m^2'):
    # Import required lattice builder
    if surface.startswith('bcc'):
        from ase.lattice.cubic import BodyCenteredCubic as lattice_builder
    elif surface.startswith('fcc'):
        from ase.lattice.cubic import FaceCenteredCubic as lattice_builder #untested
    elif surface.startswith('diamond'):
        from ase.lattice.cubic import Diamond as lattice_builder #untested
    ## Append other lattice builders here
    else:
        print('Error: Unsupported lattice ordering.')

    # Set orthogonal directions for cell axes
    if surface.endswith('100'):
        directions=[[1,0,0], [0,1,0], [0,0,1]] #tested for bcc
    elif surface.endswith('110'):
        directions=[[1,1,0], [-1,1,0], [0,0,1]] #tested for bcc
    elif surface.endswith('111'):
        directions=[[1,1,1], [-2,1,1],[0,-1,1]] #tested for bcc
    ## Append other cell axis options here
    else:
        print('Error: Unsupported surface orientation.')
    
    # Make bulk and slab with same number of atoms (size)
    bulk = lattice_builder(directions=directions, size=size, symbol=symbol, latticeconstant=a0, pbc=(1,1,1))
    cell = bulk.get_cell() ; cell[0,:] *=2 # vacuum along x axis (surface normal)
    slab = bulk.copy() ; slab.set_cell(cell)
    
    # Optimize the geometries
    from ase.optimize import LBFGSLineSearch
    bulk.calc = calc ; opt_bulk = LBFGSLineSearch(bulk) ; opt_bulk.run(fmax=fmax)
    slab.calc = calc ; opt_slab = LBFGSLineSearch(slab) ; opt_slab.run(fmax=fmax)

    # Find surface energy
    import numpy as np
    Ebulk = bulk.get_potential_energy() ; Eslab = slab.get_potential_energy()
    area = np.linalg.norm(np.cross(slab.get_cell()[1,:],slab.get_cell()[2,:]))
    gamma_ase = (Eslab - Ebulk)/(2*area)

    # Convert to required units
    if unit == 'ASE':
        return [gamma_ase,'ase_units']
    else:
        from ase import units
        gamma_SI = (gamma_ase / units.J ) * (units.m)**2
        if unit =='J/m^2':
            return [gamma_SI,'J/m^2']
        elif unit == '0.1J/m^2':
            return [10*gamma_SI,'0.1J/m^2'] # units required for the fracture code
        else:
            print('Error: Unsupported unit of surface energy.')

In [54]:
surface_energy, unit = find_surface_energy(el, calc, a0, 'diamond111')

                 Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGSLineSearch:    0 15:49:01     -416.313600*       0.0000
                 Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGSLineSearch:    0 15:49:01     -403.303800*       0.0000


In [ ]:
surface_energy  = 1.08  * 10    # GPa*A = 0.1 J/m^2

# Crack system
n               = [ 4, 6, 1 ]
crack_surface   = [ 1,-1, 0 ]
crack_front     = [ 1, 1, 0 ]
crack_tip       = [ 41, 56 ]
skin_x, skin_y = 1, 1

vacuum          = 6.0

# Simulation control
nsteps          = 31
# Increase stress intensity factor
k1              = np.linspace(0.8, 1.5, nsteps)
# Don't move crack tip
tip_dx          = np.zeros_like(k1)
tip_dz          = np.zeros_like(k1)

fmax            = 0.05

# Setup crack system
cryst = diamond(el, a0, n, crack_surface, crack_front)
set_groups(cryst, n, skin_x, skin_y)

ase.io.write('cryst.cfg', cryst)

# Compute crack tip position
r0 = np.sum(cryst.get_positions()[crack_tip,:], axis=0)/len(crack_tip)
tip_x0, tip_y0, tip_z0 = r0